In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/ten

In [5]:
from prepro_utils import preprocess_text, encode_ids, encode_pieces
import sentencepiece as spm

sp_model = spm.SentencePieceProcessor()
sp_model.Load('tiny-bert-v1/sp10m.cased.bert.model')

with open('tiny-bert-v1/sp10m.cased.bert.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [6]:
from glob import glob
import json

left, right, label = [], [], []
for file in glob('../text-similarity/*k.json'):
    with open(file) as fopen:
        x = json.load(fopen)
    for i in x:
        splitted = i[0].split(' <> ')
        if len(splitted) != 2:
            continue
        left.append(splitted[0])
        right.append(splitted[1])
        label.append(i[1])

In [7]:
list(zip(left[:10], right[:10], label[:10]))

[('Bagaimanakah saya boleh menjadi ahli geologi yang baik?',
  'Apa yang perlu saya lakukan untuk menjadi ahli geologi yang hebat?',
  1),
 ('Sekiranya saya membeli tiago?',
  'Apa yang menjaga kanak-kanak aktif dan jauh dari telefon dan permainan video?',
  0),
 ('Motorola (syarikat): Bolehkah saya menggodam Piagam Motorolla DCX3400 saya?',
  'Bagaimana saya mengesan Motorola DCX3400 untuk internet percuma?',
  0),
 ('Apakah kisah Kohinoor (Koh-i-Noor) Diamond?',
  'Apa yang akan berlaku jika kerajaan India mencuri kembali berlian Kohinoor (Koh-i-Noor)?',
  0),
 ('Apakah panduan langkah demi langkah untuk melabur dalam pasaran saham di india?',
  'Apakah panduan langkah demi langkah untuk melabur dalam pasaran saham?',
  0),
 ('Bilakah anda menggunakan シ bukan し?',
  'Bilakah anda menggunakan "&" bukan "dan"?',
  0),
 ('Astrologi: Saya Capricorn Sun Cap moon dan cap naik ... apa kata itu tentang saya?',
  'Saya Capricorn tiga kali ganda (Sun, Moon dan naik di Capricorn) Apa kata ini t

In [8]:
l = {'contradiction': 0, 'entailment': 1}


snli = glob('../text-similarity/part*.json')
for file in snli:
    with open(file) as fopen:
        x = json.load(fopen)
    for i in x:
        splitted = i[1].split(' <> ')
        if len(splitted) != 2:
            continue
        if i[0] not in l:
            continue
        left.append(splitted[0])
        right.append(splitted[1])
        try:
            label.append(l[i[0]])
        except Exception as e:
            print(e)
            print(splitted, i[0])

In [9]:
list(zip(left[-10:], right[-10:], label[-10:]))

[('Seorang wanita tidur di bawah selimut biru dan putih di dalam kereta.',
  'Seorang wanita tidur di dalam kereta.',
  1),
 ('Sesetengah orang di dalam air dan seorang yang berdiri di atas papan.',
  'Orang-orang di pantai.',
  1),
 ('Sesetengah orang di dalam air dan seorang yang berdiri di atas papan.',
  'Orang ramai sedang bersiap untuk tidur.',
  0),
 ('Lelaki dalam pakaian polis bersebelahan seorang lelaki dalam kostum zombie.',
  'Wanita dalam pakaian polis bersebelahan dengan seorang lelaki dalam kostum zombie.',
  0),
 ('Lelaki dalam pakaian polis bersebelahan seorang lelaki dalam kostum zombie.',
  'Seseorang dalam pakaian polis bersebelahan seorang lelaki dalam kostum zombie.',
  1),
 ('Seorang lelaki dalam pakaian seragam berdiri di sebelah seorang lelaki dalam kostum seram.',
  'Lelaki duduk di sebelah wanita.',
  0),
 ('Di sebalik dermaga yang panjang ke dalam tasik, beberapa orang berenang dan menikmati air.',
  'Cuaca sangat buruk di luar.',
  0),
 ('Seorang wanita tid

In [10]:
mnli = glob('../text-similarity/translated-*.json')
mnli

['../text-similarity/translated-1.json',
 '../text-similarity/translated-4.json',
 '../text-similarity/translated-3.json',
 '../text-similarity/translated-0.json',
 '../text-similarity/translated-2.json']

In [11]:
for file in mnli:
    with open(file) as fopen:
        x = json.load(fopen)
    for i in x:
        if len(i) != 3:
            continue
        splitted = i[2].split(' <> ')
        if len(splitted) != 3:
            continue
        if i[1] not in l:
            continue
        left.append(splitted[0])
        right.append(splitted[1])
        try:
            label.append(l[i[1]])
        except Exception as e:
            print(e)
            print(splitted, i)

In [12]:
list(zip(left[-10:], right[-10:], label[-10:]))

[('Vista yang mengagumkan ini pada awalnya dirancang untuk dilihat oleh Napo Leon dari bilik tidurnya di Louvre, yang pada masa itu adalah istana.',
  'Pemandangan monumental dibina untuk dilihat oleh Napoleon dari bilik tidurnya di Louvre.',
  1),
 ('Ia menggeram dan tersentak di udara, dan walaupun berjaya menghentak satu korban (yang, dengan sentuhan yang bagus, melekat di bahagian bawah kakinya seperti sepotong permen karet), secara amnya ia bergerak dengan gaya berjalan yang berat sehingga kita mungkin juga kembali pada tahun 60-an menyaksikan Lembah Gwangi. Di manakah kelajuan maut pemangsa?',
  'Pemangsa tidak pernah dapat membunuh mangsa.',
  0),
 ('Republik Turki moden bermula pada tahun 1923, tetapi sejarah tanah di dalam sempadannya bermula sejak awal umat manusia.',
  'Turki mempunyai sejarah panjang sejak ribuan tahun.',
  0),
 ('Kawasan membeli-belah utama adalah Tsim Sha Tsui di Kowloon, terutama di sepanjang Jalan Nathan; Pusat di Pulau Hong Kong, terutama untuk barang 

In [13]:
BERT_INIT_CHKPNT = 'tiny-bert-v1/model.ckpt'
BERT_CONFIG = 'tiny-bert-v1/config.json'

In [14]:
from tqdm import tqdm
MAX_SEQ_LENGTH = 300

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
              tokens_a.pop()
        else:
              tokens_b.pop()
                
def get_inputs(left, right):

    input_ids, input_masks, segment_ids = [], [], []
    for i in tqdm(range(len(left))):
        tokens_a = tokenizer.tokenize(left[i])
        tokens_b = tokenizer.tokenize(right[i])
        _truncate_seq_pair(tokens_a, tokens_b, MAX_SEQ_LENGTH - 3)

        tokens = []
        segment_id = []
        tokens.append("[CLS]")
        segment_id.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_id.append(0)
        tokens.append("[SEP]")
        segment_id.append(0)
        for token in tokens_b:
            tokens.append(token)
            segment_id.append(1)
        tokens.append("[SEP]")
        segment_id.append(1)
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)
        while len(input_id) < MAX_SEQ_LENGTH:
            input_id.append(0)
            input_mask.append(0)
            segment_id.append(0)

        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    return input_ids, input_masks, segment_ids

In [15]:
len(left), len(right)

(1041526, 1041526)

In [16]:
input_ids, input_masks, segment_ids = get_inputs(left, right)

100%|██████████| 1041526/1041526 [04:09<00:00, 4182.64it/s]


In [17]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
epoch = 20
batch_size = 128
warmup_proportion = 0.1
num_train_steps = int(len(left) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [18]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=training,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        self.logits = tf.layers.dense(output_layer, dimension_output)
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from tiny-bert-v1/model.ckpt


In [20]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, label, test_size = 0.2)

In [21]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_masks = test_input_masks[i: index]
        batch_segment = test_segment_ids[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/6510 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.785645
time taken: 2713.207044363022
epoch: 0, training loss: 0.524101, training acc: 0.725539, valid loss: 0.445311, valid acc: 0.785645



train minibatch loop:   0%|          | 0/6510 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.785645, current acc: 0.825923
time taken: 2713.2396500110626
epoch: 1, training loss: 0.405105, training acc: 0.810648, valid loss: 0.380279, valid acc: 0.825923



train minibatch loop:   0%|          | 0/6510 [00:00<?, ?it/s]

...

epoch: 7, pass acc: 0.863426, current acc: 0.864474
time taken: 2708.3127732276917
epoch: 7, training loss: 0.234747, training acc: 0.902913, valid loss: 0.329308, valid acc: 0.864474



train minibatch loop:   0%|          | 0/6510 [00:00<?, ?it/s]

epoch: 8, pass acc: 0.864474, current acc: 0.865158
time taken: 2707.6398663520813
epoch: 8, training loss: 0.221076, training acc: 0.909245, valid loss: 0.331831, valid acc: 0.865158



test minibatch loop: 100%|██████████| 1628/1628 [04:33<00:00,  5.95it/s, accuracy=0.9, cost=0.35]   

time taken: 2707.5254366397858
epoch: 9, training loss: 0.208686, training acc: 0.915261, valid loss: 0.345386, valid acc: 0.864755

break epoch:10



In [22]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'tiny-bert-similarity/model.ckpt')

'tiny-bert-similarity/model.ckpt'

In [23]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate,
    training = False
)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'tiny-bert-similarity/model.ckpt')

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Restoring parameters from tiny-bert-similarity/model.ckpt


In [24]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'Placeholder_3',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/Softmax',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/output/dense/kernel',
 'bert/encoder/layer_0/output/dense/bias',
 'bert/encoder/layer_0/output/LayerNorm/gamma',
 'bert/encoder/lay

In [25]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_masks = test_input_masks[i: index]
    batch_segment = test_segment_ids[i: index]
    batch_y = test_Y[i: index]
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 1628/1628 [03:18<00:00,  8.21it/s]


In [26]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['not similar', 'similar'],
        digits = 5
    )
)

              precision    recall  f1-score   support

 not similar    0.90845   0.85704   0.88200    114843
     similar    0.83576   0.89387   0.86384     93463

    accuracy                        0.87357    208306
   macro avg    0.87210   0.87546   0.87292    208306
weighted avg    0.87583   0.87357   0.87385    208306



In [38]:
test_ids, test_masks, test_segment = get_inputs(['rakyat mahu tukar kerajaan'], 
                                                 ['teks ini adalah mengenai najib'])

100%|██████████| 1/1 [00:00<00:00, 1290.56it/s]


In [39]:
sess.run(
    model.logits,
    feed_dict = {
        model.X: test_ids,
        model.segment_ids: test_segment,
        model.input_masks: test_masks
    },
)

array([[ 3.0846386, -3.0090528]], dtype=float32)

In [35]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [36]:
freeze_graph('tiny-bert-similarity', strings)

INFO:tensorflow:Restoring parameters from tiny-bert-similarity/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 73 variables.
INFO:tensorflow:Converted 73 variables to const ops.
2385 ops in the final graph.


In [37]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'tiny-bert-similarity/frozen_model.pb'
outPutname = "v36/similarity/tiny-bert-similarity.pb"

s3 = boto3.client('s3')

s3.upload_file(Key,bucketName,outPutname)